In [ ]:
import sys
from datetime import datetime, timedelta

import pypdfium2 as pdfium
import streamlit as st
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
from snowflake.snowpark.functions import col

from reportlab.lib.pagesizes import letter
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image
from reportlab.graphics.shapes import Drawing, Line

In [ ]:
# Initialize Snowflake session and set the database and schema
session = get_active_session()
session.use_database("MY_DB")
session.use_schema("MY_SCHEMA")

# Reference to the stage
stage = "@MY_STAGE"

In [ ]:
# Define the mapping for command-line argument to product details
product_mapping = {
    'a': ('Widget A', 'WIDGET_A', True),
    'b': ('Widget B', 'WIDGET_B', True),
    'c': ('Widget C', 'WIDGET_C', True),
    'test': ('TEST Widget A', 'WIDGET_A', False)
}

# Get the command-line argument (default to 'test' if not provided)
arg = sys.argv[0] if sys.argv[0] in product_mapping else 'test'

# Use the mapping to set product variables, or fallback to default values
product, db_product, prod = product_mapping.get(arg)

In [ ]:
# Get today's date and calculate related dates
today = datetime.now()
yesterday = today - timedelta(days=1)
last_week = today - timedelta(days=7)

# Format dates as strings
current_date = today.strftime('%Y-%m-%d')
yesterday_date = yesterday.strftime('%Y-%m-%d')
previous_date = last_week.strftime('%Y-%m-%d')

# Create report name
report_name = f"{current_date}_{db_product.lower()}_report"

In [ ]:
# raw data
report_df = session.table("fct_widget_sales")

widget_df = (
    report_df.select(
        'SALE_DATE',
        f"{db_product}_TOTAL_SALES_DOLLARS",
        f"{db_product}_TOTAL_SALES_UNITS"
    )
    .filter(
        col('SALE_DATE') > (previous_date)
    )
)

widget_sales_dollars_df = (
    widget_df.select(
        'SALE_DATE',
        f"{db_product}_TOTAL_SALES_DOLLARS"
        )
)

widget_sales_units_df = (
    widget_df.select(
        'SALE_DATE',
        f"{db_product}_TOTAL_SALES_UNITS"
        )
)

In [ ]:
# Define styles
styles = getSampleStyleSheet()

# Header style
header_style = ParagraphStyle(
    'Header',
    parent=styles['Heading1'],
    fontSize=16,
    alignment=1,  # Center alignment
    textColor='black'
)

# Subheader style
subheader_style = ParagraphStyle(
    'SubHeader',
    parent=styles['Normal'],
    fontSize=10,
    alignment=1,  # Center alignment
    fontName='Helvetica', 
)

# Footer style
footer_style = ParagraphStyle(
    'Footer',
    parent=subheader_style,
    alignment=0  # Left alignment
)

# Generate the PDF content
def create_pdf(temp_pdf_path, product, current_date, yesterday_date, widget_sales_dollars_df, widget_sales_units_df, db_product):
    doc = SimpleDocTemplate(
        temp_pdf_path,
        pagesize=letter,
        leftMargin=15,  # Reduced left margin
        rightMargin=15,  # Reduced right margin
        topMargin=5,  # Reduced top margin
        bottomMargin=0,  # Reduced bottom margin
    )

    content = []

    # Header
    header = Paragraph(f"{product} Sales Report - {current_date}", header_style)
    subheader1 = Paragraph(
        f"This is the <b>{product}</b> sales report which details all {product} sales from {yesterday_date} until {current_date}.",
        subheader_style
    )
    subheader2 = Paragraph(
        f"This report is automatically generated using <font color='blue'>{chr(0x2744)}</font>", subheader_style
    )
    contact_info = Paragraph(
        'Please reach out to <a href="mailto:me@email.com" color="blue">me</a> with any questions.',
        subheader_style
    )

    # Add header and subheader to content
    content.extend([header, Spacer(1, 6), subheader1, Spacer(1, 6), subheader2, Spacer(1, 6), contact_info, Spacer(1, 6)])

    # Horizontal line after subheader
    horizontal_line = Drawing(400, 15)
    horizontal_line.add(Line(0, 0, 565, 0, strokeColor=colors.black))
    content.append(horizontal_line)
    content.append(Spacer(1, 6))  # Adds space after the line

    # Add charts
    content.extend([create_sales_chart(widget_sales_dollars_df, db_product, product), Spacer(1, 6)])
    content.extend([create_units_chart(widget_sales_units_df, db_product, product), Spacer(1, 6)])

    # Footer text
    footer_text = Paragraph(
        f"This is the official WidgetCo {product.lower()} report from {current_date}.",
        footer_style
    )

    # Footer logo (optional)
    logo_image = "logo.png"  # Path to your logo file
    content.append(Image(logo_image, 35, 35, hAlign="LEFT"))
    content.append(footer_text)

    # Build the PDF
    doc.build(content)


def create_sales_chart(df, db_product, product):
    """Generates the sales chart."""
    chart_df = df.to_pandas()
    dates = chart_df['SALE_DATE']
    values = chart_df[f"{db_product}_TOTAL_SALES_DOLLARS"]
    fig, ax = plt.subplots()
    bar_container = ax.bar(dates, values, color='#008b8b')
    ax.bar_label(bar_container, fmt=lambda x: f"${x:,.2f}")
    ax.set_xticks(dates)
    ax.xaxis.set_major_formatter(DateFormatter("%Y-%m-%d"))
    plt.xticks(rotation=45, ha='right')
    ax.set_xlabel('Date')
    ax.set_ylabel('Sales ($)')
    ax.set_title(f'{product} Sales by Day')
    chart_image = "chart1.png"
    plt.savefig(chart_image, bbox_inches='tight')
    plt.close()
    return Image(chart_image, width=300, height=300)


def create_units_chart(df, db_product, product):
    """Generates the units sold chart."""
    chart_df = df.to_pandas().sort_values(by='SALE_DATE')
    dates = chart_df['SALE_DATE']
    values = chart_df[f"{db_product}_TOTAL_SALES_UNITS"]
    cumulative_values = values.cumsum()
    fig, ax = plt.subplots()
    ax.plot(dates, cumulative_values, marker='o', label='Cumulative Units Sold', color='#008b8b')
    for i, value in enumerate(cumulative_values):
        ax.text(dates.iloc[i], value, f"{value:,.0f}", fontsize=9, ha='center', va='bottom')
    ax.set_xticks(dates)
    ax.xaxis.set_major_formatter(DateFormatter("%Y-%m-%d"))
    plt.xticks(rotation=45, ha='right')
    ax.set_xlabel('Date')
    ax.set_ylabel('Total Units Sold')
    ax.set_title(f'{product} Cumulative Units Sold by Day')
    ax.grid(visible=True, linestyle='--', alpha=0.7)
    chart_image = "chart2.png"
    plt.savefig(chart_image, bbox_inches='tight')
    plt.close()
    return Image(chart_image, width=300, height=300)


In [ ]:
# Save temp PDF
create_pdf(f"{report_name}.pdf", product, current_date, yesterday_date, widget_sales_dollars_df, widget_sales_units_df, db_product)

In [ ]:
if prod == False:
    # Load the generated PDF
    pdf = pdfium.PdfDocument(f"{report_name}.pdf")
    
    # Loop over all pages, render and save each one as an image
    for i in range(len(pdf)):
        page = pdf[i]
        image = page.render(scale=4).to_pil()
        
        # Save the rendered image as a JPEG file
        image_path = f"{report_name}_{i + 1}.jpg"
        image.save(image_path)
        
        # Display the image in the Streamlit app
        st.image(image_path, caption=f"Page {i + 1}")

In [ ]:
if prod == True:
    put_result = session.file.put(
        f"{report_name}.pdf",
        f"{stage}/{db_product.lower()}_reports", 
        auto_compress=False
    )